## Imports

In [ ]:
import os
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
#!nvidia-smi

## Copy & extract the .zip file into colab workspace `/content/dataset`



In [ ]:
#os.mkdir("dataset")

In [ ]:
#!cp -av "/content/drive/MyDrive/BRACU Thesis Groups/Seizure/dataset/Copy of 5s Inter-ictal.zip" "/content/dataset"

In [ ]:
#!cp -av "/content/drive/MyDrive/BRACU Thesis Groups/Seizure/dataset/Copy of 5s preictal.zip" "/content/dataset"

In [ ]:
#!unzip "dataset/Copy of 5s Inter-ictal.zip" -d "dataset/interictal"

In [ ]:
#!unzip "dataset/Copy of 5s preictal.zip" -d "dataset/preictal"

## Create a list of dataset file directories (5s windows stored as .npy array)

In [ ]:
input_dir_interictal = "C:/Users/User/Downloads/Nabiha/Thesis/code files/all patient files/5s interictal file"
input_dir_preictal = "C:/Users/User/Downloads/Nabiha/Thesis/code files/all patient files/5s preictal file"

interictal_paths = sorted(
    [
        os.path.join(input_dir_interictal, fname)
        for fname in os.listdir(input_dir_interictal)
        if fname.endswith(".npy")
    ]
)

preictal_paths = sorted(
    [
        os.path.join(input_dir_preictal, fname)
        for fname in os.listdir(input_dir_preictal)
        if fname.endswith(".npy")
    ]
)

input_img_paths = interictal_paths + preictal_paths

print("Number of samples:", len(input_img_paths))

for input_img_path in input_img_paths[:10]:
    print(input_img_path)


Number of samples: 35131
C:/Users/User/Downloads/Nabiha/Thesis/code files/all patient files/5s interictal file\chb01_03.edf___(Inter-ictal)___1.npy
C:/Users/User/Downloads/Nabiha/Thesis/code files/all patient files/5s interictal file\chb01_03.edf___(Inter-ictal)___10.npy
C:/Users/User/Downloads/Nabiha/Thesis/code files/all patient files/5s interictal file\chb01_03.edf___(Inter-ictal)___100.npy
C:/Users/User/Downloads/Nabiha/Thesis/code files/all patient files/5s interictal file\chb01_03.edf___(Inter-ictal)___101.npy
C:/Users/User/Downloads/Nabiha/Thesis/code files/all patient files/5s interictal file\chb01_03.edf___(Inter-ictal)___102.npy
C:/Users/User/Downloads/Nabiha/Thesis/code files/all patient files/5s interictal file\chb01_03.edf___(Inter-ictal)___103.npy
C:/Users/User/Downloads/Nabiha/Thesis/code files/all patient files/5s interictal file\chb01_03.edf___(Inter-ictal)___104.npy
C:/Users/User/Downloads/Nabiha/Thesis/code files/all patient files/5s interictal file\chb01_03.edf___(I

## Define parameters

In [ ]:
img_size = (18, 1280) #5s window shape (num_channels, 5 * Fs)
num_classes = 2
batch_size = 32

## Data loader

In [ ]:
class EEGData(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):

        """Returns tuple (input, label) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="float32")

        for j, path in enumerate(batch_input_img_paths):
            img = np.load(path)
            img = (img - img.min())/(img.max() - img.min())
            x[j] = np.expand_dims(img, axis=-1)
        y = np.zeros((self.batch_size,) + (1,), dtype="uint8")

        for j, path in enumerate(batch_input_img_paths):
            #print(path)
            data_type = path.split("/")[-1].split(" ")[1]
           # print(data_type)
            # interictal -> 0
            # preictal -> 1

            if data_type == "interictal":
                label = 0
            elif data_type == "preictal":
                label = 1
            y[j] = label
        return x, y


In [ ]:
import random

# Split our img paths into a training, validation & test set as 70%, 20% & 10%
val_samples = int(len(input_img_paths)*0.2)
test_samples = int(len(input_img_paths)*0.1)

random.Random(1337).shuffle(input_img_paths)

train_input_file_paths = input_img_paths[:-(val_samples+test_samples)]

val_input_file_paths = input_img_paths[-(val_samples+test_samples):-test_samples]

test_input_file_paths = input_img_paths[-test_samples:]

# Instantiate data Sequences for each split
train_gen = EEGData(
    batch_size, img_size, train_input_file_paths
)
val_gen = EEGData(
    batch_size, img_size, val_input_file_paths
)
test_gen = EEGData(
    batch_size, img_size, test_input_file_paths
)
print("len(test_gen)",len(test_gen))
print("len(train_gen)",len(train_gen))
print("len(val_gen)",len(val_gen))

len(test_gen) 109
len(train_gen) 768
len(val_gen) 219


## Check one sample

In [ ]:
X_sample, y_sample = train_gen[0]
print(X_sample)
print(X_sample.shape)
print(X_sample.max())
print(X_sample.min())
print(y_sample)
print(y_sample.shape)
print(y_sample.max())
print(y_sample.min())

[[[[0.38059583]
   [0.38454744]
   [0.390197  ]
   ...
   [0.3562577 ]
   [0.3632931 ]
   [0.37288317]]

  [[0.36470047]
   [0.4905302 ]
   [0.55002064]
   ...
   [0.2995669 ]
   [0.29659072]
   [0.30958015]]

  [[0.405519  ]
   [0.34946603]
   [0.26909253]
   ...
   [0.39127672]
   [0.41668767]
   [0.41805366]]

  ...

  [[0.36509237]
   [0.37547064]
   [0.40508673]
   ...
   [0.38112956]
   [0.38014165]
   [0.373625  ]]

  [[0.3789522 ]
   [0.41204354]
   [0.43062437]
   ...
   [0.41883466]
   [0.4212606 ]
   [0.4320816 ]]

  [[0.4709625 ]
   [0.47114787]
   [0.46263647]
   ...
   [0.32590872]
   [0.34530038]
   [0.36384436]]]


 [[[0.6227802 ]
   [0.6125198 ]
   [0.59757817]
   ...
   [0.54408264]
   [0.5602513 ]
   [0.59268004]]

  [[0.5400129 ]
   [0.535728  ]
   [0.56110245]
   ...
   [0.57082933]
   [0.6146771 ]
   [0.62622285]]

  [[0.675651  ]
   [0.6346833 ]
   [0.5894367 ]
   ...
   [0.6768681 ]
   [0.6398167 ]
   [0.61326265]]

  ...

  [[0.6354846 ]
   [0.6143525 ]
   [0.5

## Define the CNN architecture

In [ ]:
def get_model(img_size):

    inputs = keras.Input(shape=img_size + (1,))

    x = layers.Conv2D(32, 3, strides=1, padding="same", activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2,2))(x)

    x = layers.Conv2D(64, 3, strides=1, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2,2))(x)

    x = layers.Conv2D(128, 3, strides=1, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2,2))(x)

    x = layers.Conv2D(256, 3, strides=1, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2,2))(x)

    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dense(64, activation = "relu")(x)
    x = layers.Dropout(0.4)(x)

    x = layers.Dense(32, activation = "relu")(x)
    x = layers.Dropout(0.4)(x)

    outputs = layers.Dense(1, activation = "sigmoid")(x)

    # Define the model
    model = keras.Model(inputs, outputs)

    return model


# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

# Build model
model = get_model(img_size)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 18, 1280, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 18, 1280, 32)      320       
_________________________________________________________________
batch_normalization (BatchNo (None, 18, 1280, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 9, 640, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 640, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 640, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 320, 64)        0     

## Plot the model

In [ ]:
#  tf.keras.utils.plot_model(
#      model,   to_file="model.png",
#      show_shapes=True,
#      show_dtype=False,
#      show_layer_names=True,
# #     rankdir="TB",
# #     expand_nested=False,
# #     dpi=96,
# #     layer_range=None,
# #     show_layer_activations=False,
#  )

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


## Train the model

In [ ]:
epochs = 1 #50
callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"), # for saving the weights after each epoch
    keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True,), # for stopping the training when performance stops improving
    tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=5) # reduce learing rate by a factor of 10 when stops improving

]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.Recall(), tf.keras.metrics.Precision()
],
)
history = model.fit(
    train_gen, epochs=epochs, callbacks=callbacks, validation_data=val_gen,
)


768/768 [==============================] - 321s 417ms/step - loss: 0.5930 - accuracy: 0.6953 - recall: 0.5257 - precision: 0.6568 - val_loss: 0.5871 - val_accuracy: 0.7041 - val_recall: 0.4236 - val_precision: 0.7352


## Evaluate on test data

In [ ]:
results = model.evaluate(test_gen, batch_size=batch_size)
print("test loss, test acc, test recall, test precision:", results)

109/109 [==============================] - 10s 90ms/step - loss: 0.5862 - accuracy: 0.7122 - recall: 0.4306 - precision: 0.7432
test loss, test acc, test recall, test precision: [0.5861572623252869, 0.7121559381484985, 0.4306151568889618, 0.7432098984718323]


## Plot training profile

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]

precision = history.history["precision"]
val_precision = history.history["val_precision"]

recall = history.history["recall"]
val_recall = history.history["val_recall"]

In [ ]:
plt.figure(figsize=(20,10))

plt.subplot(2, 2, 1)
plt.plot(loss, 'r', label="train loss")
plt.plot(val_loss, 'b', label="val loss")
plt.xlabel("Epoch")
plt.ylim((0, 1))
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(accuracy, 'r', label="train accuracy")
plt.plot(val_accuracy, 'b', label="val accuracy")
plt.xlabel("Epoch")
plt.ylim((0, 1))
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(precision, 'r', label="train precision")
plt.plot(val_precision, 'b', label="val precision")
plt.xlabel("Epoch")
plt.ylim((0, 1))
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(recall, 'r', label="train recall")
plt.plot(val_recall, 'b', label="val recall")
plt.xlabel("Epoch")
plt.ylim((0, 1))
plt.legend()

plt.show()

In [ ]:
# history2 = model.fit(
#     train_gen, epochs=epochs, callbacks=callbacks, validation_data=val_gen,
# )


In [ ]:
keras.backend.clear_session()